In [4]:
import pandas as pd
from sklearn.cluster import KMeans
import yfinance as yf
from datetime import datetime, timedelta

def fetch_data(symbol):
    # Fetch historical data for the stock
    stock_data = yf.download(symbol, period="max")
    return stock_data

def cluster_data(stock_data, n_clusters):
    # Select features for clustering
    X = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
    stock_data['Cluster'] = kmeans.fit_predict(X)

    return stock_data, kmeans

def predict_cluster(kmeans, last_data_point):
    # Predict the cluster of the last data point
    future_data = [[
        last_data_point['Open'].iloc[-1], 
        last_data_point['High'].iloc[-1], 
        last_data_point['Low'].iloc[-1], 
        last_data_point['Close'].iloc[-1], 
        last_data_point['Volume'].iloc[-1]
    ]]
    predicted_cluster = kmeans.predict(future_data)[0]
    return predicted_cluster

def estimate_future_price(stock_data, predicted_cluster):
    # Calculate the average future price change for the cluster
    cluster_data = stock_data[stock_data['Cluster'] == predicted_cluster]
    cluster_data['Next_Close'] = cluster_data['Close'].shift(-1)
    cluster_data.dropna(inplace=True)
    cluster_data['Price_Change'] = cluster_data['Next_Close'] - cluster_data['Close']
    average_price_change = cluster_data['Price_Change'].mean()

    return average_price_change

def main():
    # Ask user for stock symbol
    symbol = input("Enter the stock symbol (e.g., GOOG): ").upper()
    prediction_minutes = int(input("Enter the prediction time interval in minutes (e.g., 5): "))
    a=[3,10]
    for x in a:
         # Ask user for the number of clusters
        n_clusters = x

        # Ask user for the prediction time interval in minutes
        
        prediction_delta = timedelta(minutes=prediction_minutes)

        # Fetch historical data for the chosen stock
        stock_data = fetch_data(symbol)

        # Check if data is available for the entered symbol
        if stock_data.empty:
            print("Error: Data not available for the entered stock symbol.")
            return

        # Cluster the data
        stock_data, kmeans = cluster_data(stock_data, n_clusters)

        # Fetch real-time data for the chosen stock
        stock = yf.Ticker(symbol)
        try:
            realtime_data = stock.history(period='5d', interval='1m')  # Use 5-minute intervals
        except Exception as e:
            print(f"Error fetching real-time data: {e}")
            return

        # Check if real-time data is available
        if realtime_data.empty:
            print("Error: Real-time data not available.")
            return

        # Get the latest data point
        last_data_point = realtime_data.tail(1)

        # Predict the cluster of the latest data point
        predicted_cluster = predict_cluster(kmeans, last_data_point)

        # Estimate future price change
        average_price_change = estimate_future_price(stock_data, predicted_cluster)

        # Get the current time and price
        current_time = last_data_point.index[-1]
        current_price = last_data_point['Close'].iloc[-1]

        # Calculate the predicted future price and future time
        future_price = current_price + average_price_change
        future_time = current_time + prediction_delta

        # Print the current and predicted future price
        print(f"Current Time: {current_time}")
        print(f"Current Price for {symbol}: {current_price:.2f}")
        print(f"Predicted Future Price for {symbol} at {future_time}: {future_price:.2f}")

if __name__ == "__main__":
    main()


Enter the stock symbol (e.g., GOOG): COALINDIA.NS
Enter the prediction time interval in minutes (e.g., 5): 316


[*********************100%%**********************]  1 of 1 completed
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5320\3290467835.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['Next_Close'] = cluster_data['Close'].shift(-1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5320\3290467835.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data.dropna(inplace=True)
C:\U

Current Time: 2024-08-08 10:14:00+05:30
Current Price for COALINDIA.NS: 527.65
Predicted Future Price for COALINDIA.NS at 2024-08-08 15:30:00+05:30: 527.72


[*********************100%%**********************]  1 of 1 completed


Current Time: 2024-08-08 10:14:00+05:30
Current Price for COALINDIA.NS: 527.65
Predicted Future Price for COALINDIA.NS at 2024-08-08 15:30:00+05:30: 527.58


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5320\3290467835.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['Next_Close'] = cluster_data['Close'].shift(-1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5320\3290467835.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data.dropna(inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5320\3290467835.py:38: Setti

##### 